# Stage 4 - Entity Matching

## 4.1 Readin data and previous save matcher

In [1]:
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import py_entitymatching as em

In [2]:
data_dir = './dataset/structured_data/'
A_filename = data_dir+'forbes_all_rename.csv' 
B_filename = data_dir+'nasdaq_rename.csv' 
blocked_res = data_dir+'blocking_results.csv' 
shared_fields = ['Name',"Country", 'Industry', "MarketValue"] ## These are shared fileds
# use the random forest matcher
m = em.load_object(data_dir+"rf_matcher.pkl"); # load matcher from previous matching stage.

In [3]:
# Load orignial tables
A = pd.read_csv(A_filename, encoding = "ISO-8859-1" );
em.set_key(A, 'id');
A1 = A[['id'] + shared_fields];
em.set_key(A1, 'id');
B = pd.read_csv(B_filename,  encoding = "ISO-8859-1");
em.set_key(B, 'id');
B1 = B[['id'] + shared_fields];
em.set_key(B1, 'id');
# Load the pre-labeled data
S1 = em.read_csv_metadata(blocked_res, 
                         key='_id',
                         ltable=A1, rtable=B1, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id',  encoding = "ISO-8859-1");

Metadata file is not present in the given path; proceeding to read the csv file.


## 4.2 Entity matching

Build features.

In [4]:
# Generate a set of features
F = em.get_features_for_matching(A1, B1)
# Add some new feature to F
def MarketValue_ratio(ltuple, rtuple) :
    try :
        return float(ltuple.MarketValue) / float(rtuple.MarketValue)
    except ValueError :
        return 0
em.add_blackbox_feature(F, 'MarketValue_ratio', MarketValue_ratio)

True

In [5]:
F.feature_name

0                                 id_id_exm
1                                 id_id_anm
2                            id_id_lev_dist
3                             id_id_lev_sim
4                 Name_Name_jac_qgm_3_qgm_3
5             Name_Name_cos_dlm_dc0_dlm_dc0
6             Name_Name_jac_dlm_dc0_dlm_dc0
7                             Name_Name_mel
8                        Name_Name_lev_dist
9                         Name_Name_lev_sim
10                            Name_Name_nmw
11                             Name_Name_sw
12          Country_Country_jac_qgm_3_qgm_3
13      Country_Country_cos_dlm_dc0_dlm_dc0
14      Country_Country_jac_dlm_dc0_dlm_dc0
15                      Country_Country_mel
16                 Country_Country_lev_dist
17                  Country_Country_lev_sim
18                      Country_Country_nmw
19                       Country_Country_sw
20        Industry_Industry_jac_qgm_3_qgm_3
21    Industry_Industry_cos_dlm_dc0_dlm_dc0
22    Industry_Industry_jac_dlm_

Compute features for all blocked tuple pairs.

In [6]:
L1 = em.extract_feature_vecs(S1, feature_table=F, attrs_after='is_match',show_progress=False)
L1 = em.impute_table(L1, 
                    exclude_attrs=['_id', 'ltable_id', 'rtable_id',"is_match"],
                    strategy='mean')

Predict using the matcher.

In [7]:
predictions = m.predict(table=L1, exclude_attrs=['_id', 'ltable_id', 'rtable_id',"is_match"], 
                  append=True, target_attr='predicted', inplace=False)

In [8]:
predictions.to_csv(data_dir+"matching_result.csv") # save matching results to data directory

## 4.3 Find duplicates

In [9]:
import collections
fd = collections.defaultdict(list) # dictionary for forbes
nd = collections.defaultdict(list) # dictionary for nasdaq
lid = predictions["ltable_id"]
rid = predictions["rtable_id"]
match = predictions["predicted"]

# forbes dictionary
for i in range(len(lid)):
    if match[i] == 0:
        continue 
    fd[lid[i]].append(rid[i])
    nd[rid[i]].append(lid[i])


For the multiple matches for one entity, we checked the results manunally and figured such similar entities are difficult to identify even by human beings. We printed one example below.

In [10]:
S1[198:256]

,Unnamed: 0,_id,ltable_id,rtable_id,ltable_Country,ltable_Industry,ltable_MarketValue,ltable_Name,rtable_Name,rtable_Country,rtable_Sector,rtable_Industry,rtable_MarketValue,is_match
198,4327,4327,1907,556,United States,Investment Services,60400,BlackRock,BlackRock Capital Investment Corporation,United States,n/a,n/a,549.696980,0
199,4344,4344,1907,557,United States,Investment Services,60400,BlackRock,Blackrock Core Bond Trust,United States,n/a,n/a,720.033932,0
200,4355,4355,1907,558,United States,Investment Services,60400,BlackRock,"Blackrock Corporate High Yield Fund, Inc.",United States,n/a,n/a,1382.468375,0
201,4366,4366,1907,559,United States,Investment Services,60400,BlackRock,BlackRock Credit Allocation Income Trust,United States,n/a,n/a,1422.440317,0
202,4371,4371,1907,560,United States,Investment Services,60400,BlackRock,"Blackrock Debt Strategies Fund, Inc.",United States,n/a,n/a,717.746746,0
203,4381,4381,1907,561,United States,Investment Services,60400,BlackRock,Blackrock Defined Opportunity Credit Trust,United States,n/a,n/a,124.084243,0
204,4415,4415,1907,562,United States,Investment Services,60400,BlackRock,BlackRock Energy and Resources Trust,United States,n/a,n/a,410.098232,0
205,4431,4431,1907,563,United States,Investment Services,60400,BlackRock,Blackrock Enhanced Equity Dividend Trust,United States,n/a,n/a,1615.764653,0
206,4438,4438,1907,564,United States,Investment Services,60400,BlackRock,"Blackrock Enhanced Government Fund, Inc",United States,n/a,n/a,85.866280,0
207,4439,4439,1907,565,United States,Investment Services,60400,BlackRock,Blackrock Floating Rate Income Strategies Fund Inc,United States,n/a,n/a,539.498751,0


Thus, we choose to only care about one-to-one matches, resulting in a total number of 704 matched entities.

In [11]:
## list of one-to-one matches
total_match = list() # list((l_id, r_id))

for ltable_id in fd :
    if len(fd[ltable_id]) == 1 :
        rtable_id = fd[ltable_id][0]
        if len(nd[rtable_id]) == 1 :
            total_match.append((ltable_id, rtable_id))

print (len(total_match))

703


## 4.4 New schema for combined table

First, we check the existing schema.

- Fileds shared with both tables

In [12]:
shared_fields

['Name', 'Country', 'Industry', 'MarketValue']

- Fields only in table A (Forbes)

In [13]:
A_unique_fields = list(set(A.columns) - set(B.columns))
A_unique_fields

['Assets', 'Employee', 'Sales', 'Profits']

- Fields only in table B (NASDAQ)

In [14]:
B_unique_fields = list(set(B.columns) - set(A.columns))
B_unique_fields

['IPOyear', 'Symbol', 'LastSale', 'Summary Quote', 'Sector']

Based on the features in two tables, we define the new table schema to be the combine of all fields above, plus 'ltable_id' and 'rtable_id' as foreign key to original tables A and B.

In [15]:
combined_fields = shared_fields + A_unique_fields + B_unique_fields
E_fields = ['ltable_id', 'rtable_id'] + combined_fields
E_fields

['ltable_id',
 'rtable_id',
 'Name',
 'Country',
 'Industry',
 'MarketValue',
 'Assets',
 'Employee',
 'Sales',
 'Profits',
 'IPOyear',
 'Symbol',
 'LastSale',
 'Summary Quote',
 'Sector']

## 4.5 Combining

Because NASDAQ table contains the company information formally registered on file, we use feature values from NASDAQ table if there is a conflict between two tables over shared fields (company name, industry, country).

In [16]:
A_fields = shared_fields + A_unique_fields
B_fields = shared_fields + B_unique_fields

In [17]:
A_indexed = A.set_index('id')
B_indexed = B.set_index('id')

In [36]:
E_match = pd.DataFrame(index=range(len(total_match)), columns=E_fields)
for i, (ltable_id, rtable_id) in enumerate(total_match) :
    E_match.loc[i, 'ltable_id'] = ltable_id
    E_match.loc[i, 'rtable_id'] = rtable_id
    E_match.loc[i] = E_match.loc[i].combine_first(B_indexed.ix[rtable_id][B_fields]).combine_first(A_indexed.ix[ltable_id][A_fields])

E_match.head()

,ltable_id,rtable_id,Name,Country,Industry,MarketValue,Assets,Employee,Sales,Profits,IPOyear,Symbol,LastSale,Summary Quote,Sector
0,2051,3485,Principal Financial Group Inc,United States,Accident &Health Insurance,18142.7,218700,NaN,11900,1200,2001,PFG,63.11,http://www.nasdaq.com/symbol/pfg,Finance
1,2053,3517,"Prudential Financial, Inc.",United States,Life Insurance,45911.9,757400,NaN,53200,5600,2001,PRU,106.68,http://www.nasdaq.com/symbol/pru,Finance
2,684,1103,Constellation Brands Inc,United States,Beverages (Production/Distribution),31775.4,17000,NaN,6500,1100,NaN,STZ,162.07,http://www.nasdaq.com/symbol/stz,Consumer Non-Durables
3,2061,4234,Torchmark Corporation,United States,Life Insurance,9082.82,19900,NaN,3800,527,NaN,TMK,77.04,http://www.nasdaq.com/symbol/tmk,Finance
4,19,372,Associated Banc-Corp,United States,Major Banks,3718.84,28200,NaN,1100,183,NaN,ASB,24.4,http://www.nasdaq.com/symbol/asb,Finance


In [37]:
unique_ltable_ids = [i for i in A.id if i not in set(ltable_id for ltable_id, _ in total_match)]
E_A_only = pd.DataFrame(index=range(len(unique_ltable_ids)), columns=E_fields)
for i, ltable_id in enumerate(unique_ltable_ids) :
    E_A_only.loc[i, 'ltable_id'] = ltable_id
    E_A_only.loc[i, A_fields] = A_indexed.ix[ltable_id][A_fields]

E_A_only.head()

,ltable_id,rtable_id,Name,Country,Industry,MarketValue,Assets,Employee,Sales,Profits,IPOyear,Symbol,LastSale,Summary Quote,Sector
0,1,NaN,77 Bank,Japan,Banks,1400,69100,NaN,853,165,NaN,NaN,NaN,NaN,NaN
1,2,NaN,Abu Dhabi Commercial Bank,United Arab Emirates,Banks,11000,62100,NaN,2800,1300,NaN,NaN,NaN,NaN,NaN
2,3,NaN,Abu Dhabi Islamic Bank,United Arab Emirates,Banks,3800,24300,NaN,1600,434,NaN,NaN,NaN,NaN,NaN
3,4,NaN,Agricultural Bank of China,China,Banks,152700,2739800,NaN,131900,28800,NaN,NaN,NaN,NaN,NaN
4,5,NaN,Ahli United Bank,Bahrain,Banks,4200,34000,NaN,1400,524,NaN,NaN,NaN,NaN,NaN


In [38]:
unique_rtable_ids = [i for i in B.id if i not in set(rtable_id for _, rtable_id in total_match)]
E_B_only = pd.DataFrame(index=range(len(unique_rtable_ids)), columns=E_fields)
for i, rtable_id in enumerate(unique_rtable_ids) :
    E_B_only.loc[i, 'rtable_id'] = rtable_id
    E_B_only.loc[i, B_fields] = B_indexed.ix[rtable_id][B_fields]

E_B_only.head()

,ltable_id,rtable_id,Name,Country,Industry,MarketValue,Assets,Employee,Sales,Profits,IPOyear,Symbol,LastSale,Summary Quote,Sector
0,NaN,1,"1-800 FLOWERS.COM, Inc.",United States,Other Specialty Stores,665.526,NaN,NaN,NaN,NaN,1999,FLWS,10.15,http://www.nasdaq.com/symbol/flws,Consumer Services
1,NaN,2,"1347 Property Insurance Holdings, Inc.",United States,Property-Casualty Insurers,48.8455,NaN,NaN,NaN,NaN,2014,PIH,8.2,http://www.nasdaq.com/symbol/pih,Finance
2,NaN,3,180 Degree Capital Corp.,United States,Finance/Investors Services,44.8111,NaN,NaN,NaN,NaN,NaN,TURN,1.45,http://www.nasdaq.com/symbol/turn,Finance
3,NaN,4,1st Constitution Bancorp (NJ),United States,Savings Institutions,148.506,NaN,NaN,NaN,NaN,NaN,FCCY,18.5,http://www.nasdaq.com/symbol/fccy,Finance
4,NaN,5,1st Source Corporation,United States,Major Banks,1262.61,NaN,NaN,NaN,NaN,NaN,SRCE,46.95,http://www.nasdaq.com/symbol/srce,Finance


In [39]:
E = E_match.append(E_A_only).append(E_B_only).reset_index(drop=True)
E.tail()

,ltable_id,rtable_id,Name,Country,Industry,MarketValue,Assets,Employee,Sales,Profits,IPOyear,Symbol,LastSale,Summary Quote,Sector
7116,NaN,4710,ZTO Express (Cayman) Inc.,China,Trucking Freight/Courier Services,9574.11,NaN,NaN,NaN,NaN,2016,ZTO,13.09,http://www.nasdaq.com/symbol/zto,Transportation
7117,NaN,4711,Zumiez Inc.,United States,Clothing/Shoe/Accessory Stores,456.463,NaN,NaN,NaN,NaN,2005,ZUMZ,18.3,http://www.nasdaq.com/symbol/zumz,Consumer Services
7118,NaN,4712,"Zweig Fund, Inc. (The)",United States,NaN,180.265,NaN,NaN,NaN,NaN,1986,ZF,11.12,http://www.nasdaq.com/symbol/zf,NaN
7119,NaN,4713,"Zynerba Pharmaceuticals, Inc.",United States,Major Pharmaceuticals,265.618,NaN,NaN,NaN,NaN,2015,ZYNE,20.1,http://www.nasdaq.com/symbol/zyne,Health Care
7120,NaN,4714,Zynga Inc.,United States,EDP Services,2474.38,NaN,NaN,NaN,NaN,2011,ZNGA,2.85,http://www.nasdaq.com/symbol/znga,Technology


In [40]:
E.to_csv(data_dir + 'E.csv')

## 4.6 Statistics of Table E

- Schema

In [41]:
E.columns

Index(['ltable_id', 'rtable_id', 'Name', 'Country', 'Industry', 'MarketValue',
       'Assets', 'Employee', 'Sales', 'Profits', 'IPOyear', 'Symbol',
       'LastSale', 'Summary Quote', 'Sector'],
      dtype='object')

- Number of tuples

In [42]:
len(E)

7121

- Example tuples

 1. Tuples from both tables A and B
 1. Tuples from only table A
 1. Tuples from only Table B

In [43]:
E.loc[[1, 563]]

,ltable_id,rtable_id,Name,Country,Industry,MarketValue,Assets,Employee,Sales,Profits,IPOyear,Symbol,LastSale,Summary Quote,Sector
1,2053,3517,"Prudential Financial, Inc.",United States,Life Insurance,45911.9,757400,NaN,53200,5600,2001,PRU,106.68,http://www.nasdaq.com/symbol/pru,Finance
563,1508,1930,"Golub Capital BDC, Inc.",United States,Food Markets,1097.56,NaN,20626,3400,NaN,2010,GBDC,19.87,http://www.nasdaq.com/symbol/gbdc,NaN


In [44]:
E.loc[[1000, 1234]]

,ltable_id,rtable_id,Name,Country,Industry,MarketValue,Assets,Employee,Sales,Profits,IPOyear,Symbol,LastSale,Summary Quote,Sector
1000,316,NaN,China Biologic Products,Hong Kong,NaN,2941,NaN,NaN,297,87,NaN,NaN,NaN,NaN,NaN
1234,588,NaN,Inditex,Spain,Apparel/Footwear Retail,103200,18800,NaN,23100,3200,NaN,NaN,NaN,NaN,NaN


In [46]:
E.loc[[5000, 7000]]

,ltable_id,rtable_id,Name,Country,Industry,MarketValue,Assets,Employee,Sales,Profits,IPOyear,Symbol,LastSale,Summary Quote,Sector
5000,NaN,2217,"Innoviva, Inc.",United States,Major Pharmaceuticals,1509.16,NaN,NaN,NaN,NaN,NaN,INVA,13.82,http://www.nasdaq.com/symbol/inva,Health Care
7000,NaN,4570,"Western Asset Managed Municipals Fund, Inc.",United States,NaN,592.45,NaN,NaN,NaN,NaN,1992,MMU,13.76,http://www.nasdaq.com/symbol/mmu,NaN


## 4.7 Merging function explained

We use NASDAQ as major table and Forbes as minor table. That is, we will proriorly use values from table B for shared fields if the field is present in both tables.

This can be best seen with those matches whose shared fields have N/A in both of the original tables A and B. We list match ids here.

In [23]:
for i_match, (ltable_id, rtable_id) in enumerate(total_match) :
    ltuple, rtuple = A_indexed.ix[ltable_id], B_indexed.ix[rtable_id]
    if any(ltuple[shared_fields].isnull()) and any(rtuple[shared_fields].isnull()) :
        print(i_match)

563


Here is a pair of tuples from A and B.

In [24]:
i_match = 563
total_match[i_match]

(1508, 1930)

In [25]:
ltable_id, rtable_id = total_match[i_match]
ltuple, rtuple = A_indexed.ix[ltable_id], B_indexed.ix[rtable_id]

In [26]:
ltuple.ix[A_fields]

Name                   Golub
Country        United States
Industry        Food Markets
MarketValue              NaN
Assets                   NaN
Employee               20626
Sales                   3400
Profits                  NaN
Name: 1508, dtype: object

In [27]:
rtuple.ix[B_fields]

Name                       Golub Capital BDC, Inc.
Country                              United States
Industry                                       NaN
MarketValue                                1097.56
IPOyear                                       2010
Symbol                                        GBDC
LastSale                                     19.87
Summary Quote    http://www.nasdaq.com/symbol/gbdc
Sector                                         NaN
Name: 1930, dtype: object

In [28]:
combined_tuple = pd.Series(index=E_fields)

In [29]:
combined_tuple = combined_tuple.combine_first(rtuple[B_fields])
combined_tuple

Assets                                         NaN
Country                              United States
Employee                                       NaN
IPOyear                                       2010
Industry                                       NaN
LastSale                                     19.87
MarketValue                                1097.56
Name                       Golub Capital BDC, Inc.
Profits                                        NaN
Sales                                          NaN
Sector                                         NaN
Summary Quote    http://www.nasdaq.com/symbol/gbdc
Symbol                                        GBDC
ltable_id                                      NaN
rtable_id                                      NaN
dtype: object

In [30]:
combined_tuple = combined_tuple.combine_first(ltuple[A_fields])
combined_tuple

Assets                                         NaN
Country                              United States
Employee                                     20626
IPOyear                                       2010
Industry                              Food Markets
LastSale                                     19.87
MarketValue                                1097.56
Name                       Golub Capital BDC, Inc.
Profits                                        NaN
Sales                                         3400
Sector                                         NaN
Summary Quote    http://www.nasdaq.com/symbol/gbdc
Symbol                                        GBDC
ltable_id                                      NaN
rtable_id                                      NaN
dtype: object

Note that we use value from table A for "Industry" which only presents in A, value from table B for "MarketValue" which only presents in B, and value from table B for "Name" which presents in both A and B.

This is exactly what we got in E.

In [31]:
E_match.loc[i_match]

ltable_id                                     1508
rtable_id                                     1930
Name                       Golub Capital BDC, Inc.
Country                              United States
Industry                              Food Markets
MarketValue                                1097.56
Assets                                         NaN
Employee                                     20626
Sales                                         3400
Profits                                        NaN
IPOyear                                       2010
Symbol                                        GBDC
LastSale                                     19.87
Summary Quote    http://www.nasdaq.com/symbol/gbdc
Sector                                         NaN
Name: 563, dtype: object